In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carregar o dataset
data = pd.read_csv('./creditcard.csv')

# Verificar as primeiras linhas do dataset
print(data.head())

# Verificar as colunas e informações gerais
print(data.info())

# Verificar a distribuição das classes
print(data['Class'].value_counts())


   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [39]:
# Separar as features (X) e a label (y)
X = data.drop('Class', axis=1)
y = data['Class']

# Padronizar as features
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [40]:
# Dividir o dataset em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [41]:
def initialize_weights(input_size, hidden_size, output_size):
    W1 = np.random.randn(input_size, hidden_size) * 0.01
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.01
    b2 = np.zeros((1, output_size))
    return W1, b1, W2, b2


In [42]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1 - sigmoid(z))


In [43]:
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    return Z1, A1, Z2, A2

def backward_propagation(X, y, Z1, A1, Z2, A2, W1, W2):
    m = X.shape[0]
    dZ2 = A2 - y.reshape(-1, 1)
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    dZ1 = np.dot(dZ2, W2.T) * sigmoid_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    return dW1, db1, dW2, db2


In [44]:
def train_mini_batch(X_train, y_train, hidden_size, epochs, learning_rate, batch_size):
    input_size = X_train.shape[1]
    output_size = 1
    W1, b1, W2, b2 = initialize_weights(input_size, hidden_size, output_size)
    
    y_train = y_train.to_numpy()

    for epoch in range(epochs):
        permutation = np.random.permutation(X_train.shape[0])
        X_train_shuffled = X_train[permutation]
        y_train_shuffled = y_train[permutation]
        
        total_cost = 0  # Inicializar o custo total para a época
        
        for i in range(0, X_train.shape[0], batch_size):
            X_batch = X_train_shuffled[i:i+batch_size]
            y_batch = y_train_shuffled[i:i+batch_size]
            
            Z1, A1, Z2, A2 = forward_propagation(X_batch, W1, b1, W2, b2)
            dW1, db1, dW2, db2 = backward_propagation(X_batch, y_batch, Z1, A1, Z2, A2, W1, W2)
            
            W1 -= learning_rate * dW1
            b1 -= learning_rate * db1
            W2 -= learning_rate * dW2
            b2 -= learning_rate * db2
            
            # Calcular o custo para o mini-batch atual e somar ao custo total
            mini_batch_cost = -np.sum(y_batch * np.log(A2) + (1 - y_batch) * np.log(1 - A2)) / X_batch.shape[0]
            total_cost += mini_batch_cost
        
        # Tirar a média do custo total para a época
        avg_cost = total_cost / (X_train.shape[0] / batch_size)
        
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Custo: {avg_cost}')
    
    return W1, b1, W2, b2




In [45]:
def predict(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_propagation(X, W1, b1, W2, b2)
    predictions = (A2 > 0.5).astype(int)
    return predictions


In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Definir parâmetros de treinamento
hidden_size = 10
epochs = 1000
learning_rate = 0.01
batch_size = 32  # Definir o tamanho do mini-batch

# Treinar o modelo usando mini-batches
W1, b1, W2, b2 = train_mini_batch(X_train, y_train, hidden_size, epochs, learning_rate, batch_size)

# Fazer previsões
y_pred = predict(X_test, W1, b1, W2, b2)
    

    

# Avaliar o desempenho
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f'Acurácia: {accuracy}')
print(f'Precisão: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')
print('Matriz de Confusão:')
print(conf_matrix)


Epoch 0, Custo: 1.0013804862484672
Epoch 100, Custo: 0.565404118605539
Epoch 200, Custo: 0.6178961276623987
Epoch 300, Custo: 0.6464877010800132
Epoch 400, Custo: 0.6667269437007781
Epoch 500, Custo: 0.6805323896449155
Epoch 600, Custo: 0.6903860444727661
Epoch 700, Custo: 0.6926291711115676
Epoch 800, Custo: 0.7114659191741732
Epoch 900, Custo: 0.7230738475210856
Acurácia: 0.9993679997191109
Precisão: 0.8163265306122449
Recall: 0.8163265306122449
F1-Score: 0.8163265306122449
Matriz de Confusão:
[[56846    18]
 [   18    80]]


## **Convergência do Modelo**

Observa-se que o custo começou em aproximadamente 1.001 e diminuiu ao longo do treinamento, mas estabilizou e até aumentou levemente, chegando a 0.723. Isso pode sugerir que o modelo pode estar se aproximando de seu limite de aprendizagem, e o aumento do custo nas últimas épocas pode ser um indício de overfitting, especialmente se o custo nos dados de validação também estivesse aumentando.

## **Desempenho do Modelo**
- A acurácia é extremamente alta, quase 99.94%. Isso indica que o modelo está acertando a maioria das previsões, o que é esperado, dada a grande quantidade de exemplos da classe majoritária (não fraudulentos).

- Precisão e o recall são de aproximadamente 81.6%. Isso é positivo, pois indica que, dos casos que o modelo previu como fraudulentos, 81.6% realmente eram fraudes

- O F1-Score de 0.816 indica um bom equilíbrio entre precisão e recall.

- A matriz de confusão mostra 80 fraudes corretamente identificadas e apenas 18 erros (falsos positivos e falsos negativos).

- Apesar do alto desbalanceamento do dataset (com 284,315 transações não fraudulentas e apenas 492 fraudulentas), a boa precisão e recall mostram que o modelo conseguiu aprender a identificar fraudes, apesar desse desbalanceamento.